<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Arquitectura Modelos  Atencionales</center></span>

<figure>
<center>
<img src="../Imagenes/Brain_model.jpg" width="600" height="400" align="center"/>
</center>
</figure>

Fuente: <a href="https://commons.wikimedia.org/wiki/File:Brain_model_12.jpg">John Campbell</a>, <a href="https://creativecommons.org/licenses/by/2.0">CC BY 2.0</a>, via Wikimedia Commons

##   <span style="color:blue">Autores</span>

- Alvaro  Montenegro, PhD, ammontenegrod@unal.edu.co
- Daniel  Montenegro, Msc, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

- Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Referencias</span> 

1. [Attn: Illustrated Attention](https://towardsdatascience.com/attn-illustrated-attention-5ec4ad276ee3#0458)
1. [Neural Machine Translation by Jointly Learning to Align and Translate (Bahdanau et. al, 2015)](https://arxiv.org/pdf/1409.0473.pdf)
1. [Effective Approaches to Attention-based Neural Machine Translation (Luong et. al, 2015)](https://arxiv.org/pdf/1508.04025.pdf)
1. [Attention Is All You Need (Vaswani et. al, 2017)](https://arxiv.org/pdf/1706.03762.pdf)
1. [Self-Attention GAN (Zhang et. al, 2018)](https://arxiv.org/pdf/1805.08318.pdf)
1. [Sequence to Sequence Learning with Neural Networks (Sutskever et. al, 2014)](https://arxiv.org/pdf/1409.3215.pdf)
1. [TensorFlow’s seq2seq Tutorial with Attention (Tutorial on seq2seq+attention)](https://github.com/tensorflow/nmt)
1. [Lilian Weng’s Blog on Attention (Great start to attention)](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html#a-family-of-attention-mechanisms)
1. [Jay Alammar’s Blog on Seq2Seq with Attention (Great illustrations and worked example on seq2seq+attention)](https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/)
1. [Google’s Neural Machine Translation System: Bridging the Gap between Human and Machine Translation (Wu et. al, 2016)](https://arxiv.org/pdf/1609.08144.pdf)


## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Modelo seq2seq](#Modelo-seq2seq)
* [Atención](#Atención)
* [Alineación](#Alineación)
* [Modelos seq2seq y atención](#Modelos-seq2seq-y-atención)
    * [Paso 0. Preparación de los estados ocultos](#Paso-0.-Preparación-de-los-estados-ocultos)
    * [Paso 1. Obtener un puntaje para cada estado oculto del encoder](#Paso-1.-Obtener-un-puntaje-para-cada-estado-oculto-del-encoder)
    * [Paso 2. Transformar todos los puntajes mediante la función softmax](#Paso-2.-Transformar-todos-los-puntajes-mediante-la-función-softmax)
    * [Paso 3. Multiplica cada estado oculto del encoder por su puntaje softmax: vectores de alineación](#Paso-3.-Multiplica-cada-estado-oculto-del-encoder-por-su-puntaje-softmax:-vectores-de-alineación)
    * [Paso 4. Suma los vectores de alineación: vector de contexto](Paso-4.-Suma-los-vectores-de-alineación:-vector-de-contexto)
    * [Paso 5. Alimenta el decodificador con el vector de contexo](#Paso-5.-Alimenta-el-decodificador-con-el-vector-de-contexo)        
    * [Entrenamiento e inferencia](#Entrenamiento-e-inferencia)
* [Ejemplos de modelos de atención](#Ejemplos-de-modelos-de-atención)
    * [Bahdanau et. al (2015)](#Bahdanau-et.-al-(2015))
    * [Luong et. al (2015)](#Luong-et.-al-(2015))
    * [Traducción automática neuronal de Google (GNMT)](#Traducción-automática-neuronal-de-Google-(GNMT))
 

## <span style="color:blue">Introducción</span>

La maquinas de aprendizaje estadístico fueron durante mucho tiempo la tendencia dominante en el área de traducción automática. Modernamente, esas máquinas han sido remplazadas por máquinas neuronales de traducción (neural machine translation o NMT). la NMT están basados en el framework seq2seq.

El framework seq2seq se basa en un autoencoder. El encoder es una red neuronal recurrente que va leyendo las palabras de entrada una por una y construye representación vectorial de cada una de estas, del mismo tamaño (embedding). El decoder es otra red neural recurrente, que condicionada sobre las entradas va generando las palabras de salida una por una, [Sequence to Sequence Learning with Neural Networks (Sutskever et. al, 2014)](https://arxiv.org/pdf/1409.3215.pdf).


El problema con este modelo básico se ilustra en la siguiente imagen. El decoder recibe el último estado oculto del encoder, el cual es en general un promedio de lo que la red ha visto en la entrada. Esto es un problema para secuencias largas, porque la red recurrente va olvidando o mas alejado llevando posiblemente a una situación catastrófica. Los mecanismos de atención fueron introducidos para resolver esta situación.

<figure>
<center>
<img src="../Imagenes/seq2seq_0.png" width="700" height="700" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Modelo clásico seq2seq.</p>
</figcaption>
</figure>


Fuente: [Attn: Illustrated Attention](https://towardsdatascience.com/attn-illustrated-attention-5ec4ad276ee3#0458)

## <span style="color:blue">Modelo seq2seq</span>

El modelo de secuencia a secuencia `seq2seq` es un modelo de aprendizaje que convierte una secuencia de entrada en una secuencia de salida. En este contexto, la secuencia es una lista de símbolos, correspondiente a las palabras en una oración. El modelo seq2seq ha logrado un gran éxito en campos como la traducción automática, los sistemas de diálogo, la respuesta a preguntas y el resumen de texto. Todas estas tareas pueden considerarse como la tarea de aprender un modelo que convierte una secuencia de entrada en una secuencia de salida. La imagen muestra la arquitectura general del modelo.


El modelo consta esencialmente de las siguientes capas.

1. Capa de incrustación de codificador
2. Capa recurrente del codificador
3. Capa de incrustación de decodificador
4. Capa recurrente del decodificador
5. Capa de salida del decodificador

La siguiente imagen ilustra la estructura del autoencoder de un modelo genérico seq2seq  en un sistema de pregunta respuesta.

<figure>
<center>
<img src="../Imagenes/seq2seq.png" width="800" height="800" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Estructura autoencoder de un modelo seq2seq de *pregunta/respuesta*.</p>
</figcaption>
</figure>

Fuente: [Write a Sequence to Sequence (seq2seq) Model](https://docs.chainer.org/en/stable/examples/seq2seq.html)

## <span style="color:blue">Atención</span>

Atención es un mecanismo que se introduce entre el codificador y el decodificador. El decodificador sigue recibiendo el estado oculto final de la secuencia de entrada como antes, mientras que el mecanismo de atención se basa en las predicciones intermedias en el estado oculto.
La imagen ilustra conceptualmente el mecanismo de atención. 


<figure>
<center>
<img src="../Imagenes/mecansimo_atencion.png" width="600" height="600" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Modelo conceptual de atención.</p>
</figcaption>
</figure>

Fuente: [Write a Sequence to Sequence (seq2seq) Model](https://docs.chainer.org/en/stable/examples/seq2seq.html)

En consecuencia el mecanismo o capa de atención es una interfaz entre el codificador y el decodificador basado en los valores intermedios del estado oculto, cada uno de los cuales está asociado a una palabra en la secuencia de entrada. El propósito del mecanismo es extraer partes útiles de la entrada que permitan alinear la entrada con  la salida. Este principio ayuda al modelo seq2seq a enfrentarse a secuencias largas.

## <span style="color:blue">Alineación</span>

En este contexto, alineación significa hacer coincidir segmentos de la entrada con segmento de la traducción, [Google’s Neural Machine Translation System: Bridging the Gap between Human and Machine Translation (Wu et. al, 2016)](https://arxiv.org/pdf/1609.08144.pdf).

Hay dos tipos de atención. La `atención global` en la cual se toman todos los estados (predicciones) del estado oculto. La `atención local` solamente toma una parte del los estados del estado oculto. En esta lección solamente consideramos atención global. Para detalles de atención local puede revisar [Effective Approaches to Attention-based Neural Machine Translation (Luong et. al, 2015)](https://arxiv.org/pdf/1508.04025.pdf).

## <span style="color:blue">Modelos seq2seq y atención</span>

El mecanismo de atención se enfoca sobre diferentes palabras asignando a cada palabra un puntaje (`score`). Estos puntajes son transformados con la función `softmax`, lo cual asigna un peso cada palabra. Con estos puntajes los estados ocultos del encoder son agregados mediante una suma pesada de los estados ocultos usando tales pesos. La construcción de una capa atencional puede subdividirse en cuatro pasos.

### <span style="color:#4CC9F0">Paso 0. Preparación de los estados ocultos</span>

Primero se preparan todos los estados ocultos del encoder y el primer estado oculto del decoder. En la siguiente imagen se supone una secuencia de entrada de cuatro palabras, por lo que hay cuatro estados ocultos (predicciones), el último de los cuales es el estado oculto  final de salida del encoder. En la imagen, los estos ocultos del encoder son los círculos en verde y el primer estado oculto del decoder se muestra en color rojo. En este ejemplo se ha asumido que el estado oculto tiene 3 componentes. Para poder hacer el emparejamiento con los estados ocultos del decoder, estos se preparan del mismo tamaño.

<figure>
<center>
<img src="../Imagenes/Paso_0_Atencion.png" width="500" height="500" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Modelo de atención. Paso 0. preparación de los estados ocultos</p>
</figcaption>
</figure>

Fuente: [Basado en Write a Sequence to Sequence (seq2seq) Model](https://docs.chainer.org/en/stable/examples/seq2seq.html)

### <span style="color:#4CC9F0">Paso 1. Obtener un puntaje para cada estado oculto del encoder</span>

El puntaje asociado para cada estado oculto del encoder se obtiene mediante el producto escalar (o producto punto) entre el valor del estado oculto y el valor del primer estado oculto del decoder. Si $hd_1 = (y_{11},\ldots, y_{1m})$ es el vector que representa el primer estado oculto del decoder y $he_i = (x_{i1},\ldots, x_{im})$ representa el i-ésimo estado oculto del encoder, entonces el puntaje asociado al estado oculto $i$ es dado por

$$
\text{score}_{i1} =<hd_1, he_i> = y_{11}x_{i1}+ \ldots + y_{1m}x_{im}, i =1,\cdots, m
$$

<figure>
<center>
<img src="../Imagenes/Paso_1_Atencion.png" width="500" height="500" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Model de atención. Paso 1. Puntajes para cada estado oculto del encoder</p>
</figcaption>
</figure>

Fuente: [Write a Sequence to Sequence (seq2seq) Model](https://docs.chainer.org/en/stable/examples/seq2seq.html)

### <span style="color:#4CC9F0">Paso 2. Transformar todos los puntajes mediante la función softmax</span>

Recuerde que con esta transformación todos los puntajes quedan en el intervalo $[0, 1]$ y suman 1.

<figure>
<center>
<img src="../Imagenes/Paso_2_Atencion.gif" width="500" height="500" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Modelo de atención. Paso 2. Los puntajes para cada estado oculto del encoder se transforman a la escala softmax</p>
</figcaption>
</figure>

Fuente: [Write a Sequence to Sequence (seq2seq) Model](https://docs.chainer.org/en/stable/examples/seq2seq.html)

### <span style="color:#4CC9F0">Paso 3. Multiplica cada estado oculto del encoder por su puntaje softmax: vectores de alineación</span>

$$
he_i \leftarrow score_i * he_i , i =1,\cdots, m
$$

El resultado se llama vector de alineación  (`alignment vector`) [Effective Approaches to Attention-based Neural Machine Translation (Luong et. al, 2015)](https://arxiv.org/pdf/1508.04025.pdf) o vector de anotación (`annotation vector`) [Neural Machine Translation by Jointly Learning to Align and Translate (Bahdanau et. al, 2015)](https://arxiv.org/pdf/1409.0473.pdf). Este el momento exacto en el cual ocurre la alineación entre la entrada y la salida.

<figure>
<center>
<img src="../Imagenes/Paso_3_Atencion.png" width="700" height="700" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Modelo de atención. Paso 3. Vectores de alineación (softmax)</p>
</figcaption>
</figure>

Fuente: [Write a Sequence to Sequence (seq2seq) Model](https://docs.chainer.org/en/stable/examples/seq2seq.html)

Revise el siguiente fragmento de código, que ilustra el calculo de los puntajes, la transformación softmax de los puntajes y el cálculo de los vectores de alineación.

In [19]:
import numpy as np
from scipy.special import softmax

encoder_hidden = np.array([[0, 2, 5],
                  [3, 5, 4],
                  [2, 0, 1],
                  [1, 1, 0]])

decoder_hidden = np.array([2, 4, 1])

score = encoder_hidden @ decoder_hidden

score_softmax = softmax(score)

alignment_vectors = np.reshape(score_softmax, (4,1))* encoder_hidden 

context_vector = np.sum(alignment_vectors, axis=0)

print('estados ocultos del encoder\n',  encoder_hidden)
print('primer estado oculto del decoder\n',  decoder_hidden)
print('puntajes para los estados ocultos del encoder\n', score)
print('puntajes estandarizados con softmax\n', np.round(score_softmax,3))
print('vectores de alineación\n', np.round(alignment_vector,3) )
print('vector de contexto\n', np.round(context_vector,3) )

estados ocultos del encoder
 [[0 2 5]
 [3 5 4]
 [2 0 1]
 [1 1 0]]
primer estado oculto del decoder
 [2 4 1]
puntajes para los estados ocultos del encoder
 [13 30  5  6]
puntajes estandarizados con softmax
 [0. 1. 0. 0.]
vectores de alineación
 [[0. 0. 0.]
 [3. 5. 4.]
 [0. 0. 0.]
 [0. 0. 0.]]
vector de contexto
 [3. 5. 4.]


Aquí vemos que la alineación para todos los estados ocultos del codificador excepto [3, 5, 4] se reducen a 0 debido a los bajos puntajes de atención. Esto significa que podemos esperar que la primera palabra traducida coincida con la palabra de entrada con la incrustación  (`embedding`) [3, 5, 4]. 

### <span style="color:#4CC9F0">Paso 4. Suma los vectores de alineación: vector de contexto</span>

Los vectores de alineación se suman para producir el `vector de contexto`, [Effective Approaches to Attention-based Neural Machine Translation (Luong et. al, 2015)](https://arxiv.org/pdf/1508.04025.pdf), [Neural Machine Translation by Jointly Learning to Align and Translate (Bahdanau et. al, 2015)](https://arxiv.org/pdf/1409.0473.pdf).

La siguiente tabla resume los resultados.

|encoder| puntaje | softmax| alineación|
|---|---|---|---|
|[0, 2, 5] | 13 | 0 | [0, 0, 0]|
|[3, 5, 4] | 30 | 1 | [3, 5, 4]|
|[2, 1, 0] | 5 | 0 | [0, 0, 0]|
|[1, 1, 0] | 6 | 0 | [0, 0, 0]|

**context** = [3, 5, 4] = [0, 0, 0] + [3, 5, 4] +  [0, 0, 0] + [0, 0, 0] 

<figure>
<center>
<img src="../Imagenes/Paso_4_Atencion.gif" width="700" height="700" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Modelo de atención. Paso 4. vector de contexto</p>
</figcaption>
</figure>

Fuente: [Write a Sequence to Sequence (seq2seq) Model](https://docs.chainer.org/en/stable/examples/seq2seq.html)

### <span style="color:#4CC9F0">Paso 5. Alimenta el decodificador con el vector de contexto</span>

La forma como se haga esto depende de la arquitectura de diseño. Vamos a ver algunas de estas arquitecturas en las siguientes secciones.

<figure>
<center>
<img src="../Imagenes/Paso_5_Atencion.png" width="700" height="700" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Modelo de atención. Paso 5. Alimenta el decodificador con el vector de contexto</p>
</figcaption>
</figure>

Fuente: [Write a Sequence to Sequence (seq2seq) Model](https://docs.chainer.org/en/stable/examples/seq2seq.html)

La siguiente imagen muestra el proceso completo en la capa de atención.

<figure>
<center>
<img src="../Imagenes/Atencion_Ilustrado_1.png" width="800" height="800" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Capa de atención completa</p>
</figcaption>
</figure>

Fuente: [Write a Sequence to Sequence (seq2seq) Model](https://docs.chainer.org/en/stable/examples/seq2seq.html)

### <span style="color:#4CC9F0">Entrenamiento e inferencia</span>

Durante la inferencia, la entrada a cada paso de tiempo $t$ del decodificador es la salida prevista del paso de tiempo $t-1$ del decodificador.
Durante el entrenamiento, la entrada a cada paso de tiempo $t$ del decodificador es nuestra salida  verdadera del paso de tiempo $t-1$ del decodificador.

## <span style="color:blue">Ejemplos de modelos de atención</span>

Hemos visto las arquitecturas de atención *seq2seq* y *seq2seq + atención* en la sección anterior. En las siguientes subsecciones, examinemos 3 arquitecturas más basadas en seq2seq para NMT (neural machine translation) que implementan la atención. Para completar, también se agregan sus puntajes de Comprensión de Evaluación Bilingüe, [BLEU](https://en.wikipedia.org/wiki/BLEU), una métrica estándar para evaluar una oración generada para una oración de referencia.

### <span style="color:#4CC9F0">Bahdanau et. al (2015)</span>

[Neural Machine Translation by Jointly Learning to Align and Translate (Bahdanau et. al, 2015)](https://arxiv.org/pdf/1409.0473.pdf). Las siguientes son cosas para tomar nota sobre la arquitectura:

1. El codificador es una unidad recurrente con compuerta bidireccional (hacia adelante + hacia atrás) (BiGRU). El decodificador es una GRU cuyo estado oculto inicial es un vector modificado del último estado oculto del GRU del codificador hacia atrás (no se muestra en el diagrama a continuación).
1. La función de puntuación en la capa de atención es el aditivo / concat.
1. La entrada al siguiente paso del decodificador es la concatenación entre la palabra generada del paso de tiempo del decodificador anterior (rosa) y el vector de contexto del paso de tiempo actual (verde oscuro).


<figure>
<center>
<img src="../Imagenes/NMT_Bahdanau.png" width="700" height="700" align="center"/>
</center>
<figcaption>
<p style="text-align:center">NMT. Bahdanau et al. Encoder: BiGRU; Decoder: GRU</p>
</figcaption>
</figure>

Fuente: [Write a Sequence to Sequence (seq2seq) Model](https://docs.chainer.org/en/stable/examples/seq2seq.html)

Los autores lograron una puntuación BLEU de 26,75 en el conjunto de datos de inglés a francés de WMT'14.

### <span style="color:#4CC9F0">Luong et. al (2015)</span>

[Effective Approaches to Attention-based Neural Machine Translation (Luong et. al, 2015)](https://arxiv.org/pdf/1508.04025.pdf). Los autores  simplifican y generalizan la arquitectura de Bahdanau et. al., como sigue

1. El codificador es una red de memoria a corto plazo (LSTM) de dos capas. El decodificador también tiene la misma arquitectura, cuyos estados ocultos iniciales son los últimos estados ocultos del codificador.
1. Las funciones de puntuación que experimentaron fueron (i) aditivo / concat, (ii) producto escalar, (iii) basado en la ubicación y (iv) "general".
1. La concatenación entre la salida del paso de tiempo del decodificador actual y el vector de contexto del paso de tiempo actual se alimentan a una red neuronal de retroalimentación (perceptron) para dar la salida final (rosa) del paso de tiempo del decodificador actual.


<figure>
<center>
<img src="../Imagenes/NMT_Luong.png" width="700" height="700" align="center"/>
</center>
<figcaption>
<p style="text-align:center">NMT. Luong et al. Encoder: Doble capa LSTM; Decoder: Doble capa LSTM</p>
</figcaption>
</figure>

Fuente: [Write a Sequence to Sequence (seq2seq) Model](https://docs.chainer.org/en/stable/examples/seq2seq.html)

### <span style="color:#4CC9F0">Traducción automática neuronal de Google (GNMT)</span>

[Google’s Neural Machine Translation System: Bridging the Gap between Human and Machine Translation (Wu et. al, 2016)](https://arxiv.org/pdf/1609.08144.pdf). Debido a que la mayoría de nosotros debe haber usado Google Translate de una forma u otra, parece imperativo hablar sobre el NMT de Google, que se implementó en 2016. GNMT es una combinación de los 2 ejemplos anteriores que hemos visto (muy inspirados en el primero).

1. El codificador consta de una pila de 8 LSTM, donde el primero es bidireccional (cuyas salidas están concatenados), y existe una conexión residual entre las salidas de capas consecutivas (a partir de la 3ª capa). El decodificador es una pila separada de 8 LSTM unidireccionales.
1. La función de puntuación utilizada es el aditivo / concat, como en [1].
1. Nuevamente, como en [Neural Machine Translation by Jointly Learning to Align and Translate (Bahdanau et. al, 2015)](https://arxiv.org/pdf/1409.0473.pdf), la entrada al siguiente paso del decodificador es la concatenación entre la salida del paso de tiempo del decodificador anterior (rosa) y el vector de contexto del paso de tiempo actual (verde oscuro).


<figure>
<center>
<img src="../Imagenes/NMT_Google.png" width="500" height="500" align="center"/>
</center>
<figcaption>
<p style="text-align:center">NMT. Google. Las conexiones de salto se indican mediante flechas curvas. Tenga en cuenta que las celdas LSTM solo muestran el estado oculto y la entrada; no muestra la entrada del estado de la celda.</p>
</figcaption>
</figure>

Fuente: [Write a Sequence to Sequence (seq2seq) Model](https://docs.chainer.org/en/stable/examples/seq2seq.html)